In [6]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

## Analysis for Finger Tapping

In [ ]:
#Mean and Standard Deviation from controls
ControlMeans = np.array([ 1.35847128,  0.14205142,  2.94011282,  0.30201834,  7.41794667,  0.66516272,
  5.10836764,  0.7627357 ,  7.12562852,  0.90081561, 11.7484118 ,  1.1699305 ,
 13.77625134,  1.40625728,  0.48690441,  0.05343162,  0.11552007,  0.11692048,
  0.10064551,  0.16363124,  0.13916518,  0.10790489,  0.110747  ,  0.1034517 ,
  2.16101563,  0.98766338,  1.10645127,  0.18796296,  0.77777778,  0.16666667])

ControlStd = np.array([ 0.8881987,   0.16891155,  2.79870573,  0.51309276, 6.69668187,  1.15180634,
  5.16647155,  1.09879943,  5.86164176,  1.200058,   10.26731521,  1.85484057,
 10.29995837,  2.0952842,   0.30585688,  0.03933395,  0.25451647,  0.21826844,
  0.20588777,  0.24649225,  0.24225607,  0.21950316,  0.2377413,   0.13248678,
  3.51881862,  1.07124939,  1.04623274,  0.16759259,  0.55555556,  0.22222222])


Data = pd.read_csv('Data_FT_longitudinal.csv', index_col=None)

#normalize features with respect to controls
Data_z = Data.copy()

features = ['MeanAmplitude', 'StdAmplitude', 'MeanSpeed', 'StdSpeed', 'MeanRMSVelocity', 'StdRMSVelocity','MeanOpeningSpeed','StdOpeningSpeed', 'MeanClosingSpeed',
            'StdClosingSpeed','MeanMaxOpeningSpeed','StdMaxOpeningSpeed','MeanMaxClosingSpeed','StdMaxClosingSpeed','MeanCycleDuration','StdCycleDuration',
            'CVAmplitude','CVSpeed','CVRMSVelocity','CVOpeningSpeed','CVClosingSpeed','CVMaxOpeningSpeed','CVMaxClosingSpeed', 'CVCycleDuration',
            'Frequency','AmplitudeDecay','VelocityDecay','RangeCycleDuration','NumberofPauses','numberofHesitations']
for feature in features:
    Data_z[feature] = (Data[feature] - ControlMeans[features.index(feature)]) / ControlStd[features.index(feature)]


In [8]:
#compare some selected features using linear mixedeffects model

# The model will include the following covariates:
# - Time (V1 or V2)
# - Age (in years)
# - Sex (male or female)
# - Handedness (left or right)
# - BaselineDiseaseDuration (in months)
# - BaselineDiseaseSeverity (baseline UPDRS III score#)

# Filter for subjects with both V1 and V2 visits
v1 = Data[Data['Visit'] == 'V1']
v2 = Data[Data['Visit'] == 'V2']
common_subjects = set(v1['Subject_ID']) & set(v2['Subject_ID'])

v1_matched = v1[v1['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
v2_matched = v2[v2['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
# Combine v1_matched and v2_matched for mixed model analysis
combined = pd.concat([v1_matched, v2_matched], ignore_index=True)


selected_features = ['MeanSpeed', 'MeanRMSVelocity', 'MeanOpeningSpeed', 'MeanClosingSpeed',
                     'Frequency', 'StdCycleDuration','RangeCycleDuration']

for feat in selected_features:
    var = feat
    model = smf.mixedlm(f"{var} ~ Time + Age + Sex + Handedness + BaselineDiseaseDuration + BaselineDiseaseSeverity ", combined, groups=combined["Subject_ID"])   
    result = model.fit()
    print(f"Mixed model results for {var}:")
    print(f"V1 mean ± std: {v1_matched[f'{var}'].mean():.2f} ± {v1_matched[f'{var}'].std():.2f}")
    print(f"V2 mean ± std: {v2_matched[f'{var}'].mean():.2f} ± {v2_matched[f'{var}'].std():.2f}")
    # print(result.summary())

    # Store p-values for Time coefficient
    if 'Time' in result.pvalues.index:
        pval_time = result.pvalues['Time']
    else:
        pval_time = np.nan

    if 'Time' in result.params.index:
        coef_time = result.params['Time']
    else:
        coef_time = np.nan

    if feat == selected_features[0]:
        pvals_time = []
        mean_diffs = []
        std_diffs = []
        percent_diffs = []

    # Compute %difference between v1 and v2
    v1_vals = v1_matched[var].values
    v2_vals = v2_matched[var].values
    diff = (v2_vals - v1_vals) / v1_vals * 100
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)

    pvals_time.append(pval_time)
    mean_diffs.append(mean_diff)
    std_diffs.append(std_diff)
    percent_diffs.append(diff)

# After loop, apply correction and print summary
if feat == selected_features[-1]:
    reject, pvals_corr, _, _ = multipletests(pvals_time, method='fdr_tsbh')
    print("\nCorrected p-values for Time coefficient (FDR):")
    for f, p, pc in zip(selected_features, pvals_time, pvals_corr):
        print(f"{f}: raw p={p:.4g}, corrected p={pc:.4g}")


Mixed model results for MeanSpeed:
V1 mean ± std: 3.15 ± 0.99
V2 mean ± std: 2.75 ± 1.03
Mixed model results for MeanRMSVelocity:
V1 mean ± std: 7.49 ± 2.27
V2 mean ± std: 6.51 ± 2.37
Mixed model results for MeanOpeningSpeed:
V1 mean ± std: 5.62 ± 1.79
V2 mean ± std: 4.80 ± 1.83
Mixed model results for MeanClosingSpeed:
V1 mean ± std: 6.50 ± 2.02
V2 mean ± std: 5.80 ± 2.17
Mixed model results for Frequency:
V1 mean ± std: 2.97 ± 0.81
V2 mean ± std: 3.01 ± 0.93
Mixed model results for StdCycleDuration:
V1 mean ± std: 0.05 ± 0.03
V2 mean ± std: 0.05 ± 0.03
Mixed model results for RangeCycleDuration:
V1 mean ± std: 0.22 ± 0.24
V2 mean ± std: 0.20 ± 0.20

Corrected p-values for Time coefficient (FDR):
MeanSpeed: raw p=0.01162, corrected p=0.0155
MeanRMSVelocity: raw p=0.008967, corrected p=0.0155
MeanOpeningSpeed: raw p=0.00465, corrected p=0.0155
MeanClosingSpeed: raw p=0.04101, corrected p=0.04101
Frequency: raw p=0.8556, corrected p=0.4889
StdCycleDuration: raw p=0.4953, corrected p=0.3

/Users/diegoguarin/anaconda3/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [9]:
#create new features as liner combinations of existing features
#based on the coefficients identified in our previous work 
# https://www.nature.com/articles/s41531-025-00999-w

#MeanSpeed, MeanRMSVelocity, MeanOpeningSpeed, MeanClosingSpeed
MovementSpeed = np.array([ 0,  0.        ,  1,  0.        ,  1,
        0.        ,  1,  0.        ,  1 ,  0.        ,
        0,  0.        ,  0,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 0,
        0.        ,  0.        ,  0.        ,  0.        , 0])
Data_z['MovementSpeed'] = Data_z[features].dot(MovementSpeed)
#StdMeanSpeed, StdRMSVelocity, StdOpeningSpeed, StdClosingSpeed
PerformanceConsistency = np.array([ 0,  0.        ,  0,  1        ,  0,
        0.        ,  1,  0.        ,  0 ,  1        ,
        0,  1        ,  0,  1       ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 0,
        0.        ,  0.        ,  0.        ,  0.        , 0])
Data_z['PerformanceConsistency'] = Data_z[features].dot(PerformanceConsistency)


#MeanAmplitude, MeanCycleDuration, StdCycleDuration, and Frequency
MovementTimingandFrequency = np.array([1, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0,
 0, 0, 0, 1,
 1, 0, 0, 0, 0, 0,
0, 0, 0, -1, 0,
0, 0, 0, 0])
Data_z['MovementTimingandFrequency'] = Data_z[features].dot(MovementTimingandFrequency)

In [10]:
#compare some selected features using linear mixedeffects model

# The model will include the following covariates:
# - Time (V1 or V2)
# - Age (in years)
# - Sex (male or female)
# - Handedness (left or right)
# - BaselineDiseaseDuration (in months)
# - BaselineDiseaseSeverity (baseline UPDRS III score#)

# Filter for subjects with both V1 and V2 visits
v1 = Data_z[Data_z['Visit'] == 'V1']
v2 = Data_z[Data_z['Visit'] == 'V2']
common_subjects = set(v1['Subject_ID']) & set(v2['Subject_ID'])

v1_matched = v1[v1['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
v2_matched = v2[v2['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
# Combine v1_matched and v2_matched for mixed model analysis
combined = pd.concat([v1_matched, v2_matched], ignore_index=True)


selected_features = ['MovementSpeed', 'PerformanceConsistency', 'MovementTimingandFrequency',]

for feat in selected_features:
    var = feat
    model = smf.mixedlm(f"{var} ~ Time + Age + Sex + Handedness + BaselineDiseaseDuration + BaselineDiseaseSeverity ", combined, groups=combined["Subject_ID"])   
    result = model.fit()
    print(f"Mixed model results for {var}:")
    print(f"V1 mean ± std: {v1_matched[f'{var}'].mean():.2f} ± {v1_matched[f'{var}'].std():.2f}")
    print(f"V2 mean ± std: {v2_matched[f'{var}'].mean():.2f} ± {v2_matched[f'{var}'].std():.2f}")
    # print(result.summary())

    # Store p-values for Time coefficient
    if 'Time' in result.pvalues.index:
        pval_time = result.pvalues['Time']
    else:
        pval_time = np.nan

    if 'Time' in result.params.index:
        coef_time = result.params['Time']
    else:
        coef_time = np.nan

    if feat == selected_features[0]:
        pvals_time = []
        mean_diffs = []
        std_diffs = []
        percent_diffs = []

    # Compute %difference between v1 and v2
    v1_vals = v1_matched[var].values
    v2_vals = v2_matched[var].values
    diff = (v2_vals - v1_vals) / v1_vals * 100
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)

    pvals_time.append(pval_time)
    mean_diffs.append(mean_diff)
    std_diffs.append(std_diff)
    percent_diffs.append(diff)

# After loop, apply correction and print summary
if feat == selected_features[-1]:
    reject, pvals_corr, _, _ = multipletests(pvals_time, method='fdr_tsbh')
    print("\nCorrected p-values for Time coefficient (FDR):")
    for f, p, pc in zip(selected_features, pvals_time, pvals_corr):
        print(f"{f}: raw p={p:.4g}, corrected p={pc:.4g}")


Mixed model results for MovementSpeed:
V1 mean ± std: 0.08 ± 1.37
V2 mean ± std: -0.49 ± 1.44
Mixed model results for PerformanceConsistency:
V1 mean ± std: 1.44 ± 1.33
V2 mean ± std: 0.68 ± 1.54
Mixed model results for MovementTimingandFrequency:
V1 mean ± std: -0.95 ± 1.49
V2 mean ± std: -1.27 ± 1.79

Corrected p-values for Time coefficient (FDR):
MovementSpeed: raw p=0.01115, corrected p=0.005574
PerformanceConsistency: raw p=0.009868, corrected p=0.005574
MovementTimingandFrequency: raw p=0.3874, corrected p=0.1291


## Analysis for Hand Movement

In [11]:
ControlMeans = np.array([0.60562566, 0.04218437, 1.46404908, 0.14366619, 3.64281644, 0.283748,
 2.96110662, 0.33293764, 2.89238806, 0.38130492, 6.1988703,  0.4602113,
 5.99320639, 0.48859037, 0.44282474, 0.045653,   0.08101462, 0.10677995,
 0.08452397, 0.12084349, 0.14351438, 0.08346807, 0.08946869, 0.09815373,
 2.38221369, 1.08052704, 1.05730708, 0.153125,   0.66666667, 0.04166667])

ControlStd = np.array([0.09528323, 0.03042973, 0.37262943, 0.06112437, 0.73162791, 0.15323251,
 0.51731929, 0.13971618, 0.70391811, 0.13534528, 0.89416578, 0.34072673,
 0.96803815, 0.25374676, 0.13639763, 0.03909477, 0.09230307, 0.0590779,
 0.05697709, 0.06904471, 0.07509828, 0.08372127, 0.06606089, 0.0665946,
 0.68357328, 0.35901232, 0.16853504, 0.17533002, 0.95278613, 0.28867513])

Data = pd.read_csv('Data_HM_longitudinal.csv', index_col=None)

#normalize features with respect to controls
Data_z = Data.copy()

features = ['MeanAmplitude', 'StdAmplitude', 'MeanSpeed', 'StdSpeed', 'MeanRMSVelocity', 'StdRMSVelocity','MeanOpeningSpeed','StdOpeningSpeed', 'MeanClosingSpeed',
            'StdClosingSpeed','MeanMaxOpeningSpeed','StdMaxOpeningSpeed','MeanMaxClosingSpeed','StdMaxClosingSpeed','MeanCycleDuration','StdCycleDuration',
            'CVAmplitude','CVSpeed','CVRMSVelocity','CVOpeningSpeed','CVClosingSpeed','CVMaxOpeningSpeed','CVMaxClosingSpeed', 'CVCycleDuration',
            'Frequency','AmplitudeDecay','VelocityDecay','RangeCycleDuration','NumberofPauses','numberofHesitations']
for feature in features:
    Data_z[feature] = (Data[feature] - ControlMeans[features.index(feature)]) / ControlStd[features.index(feature)]



In [12]:
#compare some selected features using linear mixedeffects model

# The model will include the following covariates:
# - Time (V1 or V2)
# - Age (in years)
# - Sex (male or female)
# - Handedness (left or right)
# - BaselineDiseaseDuration (in months)
# - BaselineDiseaseSeverity (baseline UPDRS III score#)

# Filter for subjects with both V1 and V2 visits
v1 = Data[Data['Visit'] == 'V1']
v2 = Data[Data['Visit'] == 'V2']
common_subjects = set(v1['Subject_ID']) & set(v2['Subject_ID'])

v1_matched = v1[v1['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
v2_matched = v2[v2['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
# Combine v1_matched and v2_matched for mixed model analysis
combined = pd.concat([v1_matched, v2_matched], ignore_index=True)


selected_features = ['MeanSpeed', 'MeanRMSVelocity', 'MeanOpeningSpeed', 'MeanClosingSpeed',
                     'Frequency', 'StdCycleDuration','RangeCycleDuration']

for feat in selected_features:
    var = feat
    model = smf.mixedlm(f"{var} ~ Time + Age + Sex + Handedness + BaselineDiseaseDuration + BaselineDiseaseSeverity ", combined, groups=combined["Subject_ID"])   
    result = model.fit()
    print(f"Mixed model results for {var}:")
    print(f"V1 mean ± std: {v1_matched[f'{var}'].mean():.2f} ± {v1_matched[f'{var}'].std():.2f}")
    print(f"V2 mean ± std: {v2_matched[f'{var}'].mean():.2f} ± {v2_matched[f'{var}'].std():.2f}")
    # print(result.summary())

    # Store p-values for Time coefficient
    if 'Time' in result.pvalues.index:
        pval_time = result.pvalues['Time']
    else:
        pval_time = np.nan

    if 'Time' in result.params.index:
        coef_time = result.params['Time']
    else:
        coef_time = np.nan

    if feat == selected_features[0]:
        pvals_time = []
        mean_diffs = []
        std_diffs = []
        percent_diffs = []

    # Compute %difference between v1 and v2
    v1_vals = v1_matched[var].values
    v2_vals = v2_matched[var].values
    diff = (v2_vals - v1_vals) / v1_vals * 100
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)

    pvals_time.append(pval_time)
    mean_diffs.append(mean_diff)
    std_diffs.append(std_diff)
    percent_diffs.append(diff)

# After loop, apply correction and print summary
if feat == selected_features[-1]:
    reject, pvals_corr, _, _ = multipletests(pvals_time, method='fdr_tsbh')
    print("\nCorrected p-values for Time coefficient (FDR):")
    for f, p, pc in zip(selected_features, pvals_time, pvals_corr):
        print(f"{f}: raw p={p:.4g}, corrected p={pc:.4g}")


Mixed model results for MeanSpeed:
V1 mean ± std: 1.47 ± 0.30
V2 mean ± std: 1.34 ± 0.35
Mixed model results for MeanRMSVelocity:
V1 mean ± std: 3.59 ± 0.64
V2 mean ± std: 3.38 ± 0.68
Mixed model results for MeanOpeningSpeed:
V1 mean ± std: 2.86 ± 0.53
V2 mean ± std: 2.74 ± 0.50
Mixed model results for MeanClosingSpeed:
V1 mean ± std: 2.89 ± 0.63
V2 mean ± std: 2.59 ± 0.78
Mixed model results for Frequency:
V1 mean ± std: 2.38 ± 0.52
V2 mean ± std: 2.24 ± 0.73
Mixed model results for StdCycleDuration:
V1 mean ± std: 0.04 ± 0.03
V2 mean ± std: 0.05 ± 0.02
Mixed model results for RangeCycleDuration:
V1 mean ± std: 0.13 ± 0.10
V2 mean ± std: 0.16 ± 0.09

Corrected p-values for Time coefficient (FDR):
MeanSpeed: raw p=0.004742, corrected p=0.009485
MeanRMSVelocity: raw p=0.01854, corrected p=0.02472
MeanOpeningSpeed: raw p=0.1459, corrected p=0.0973
MeanClosingSpeed: raw p=0.00241, corrected p=0.009485
Frequency: raw p=0.2069, corrected p=0.1183
StdCycleDuration: raw p=0.03966, corrected p

/Users/diegoguarin/anaconda3/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/diegoguarin/anaconda3/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [13]:
#create new features as liner combinations of existing features
#based on the coefficients identified in our previous work 
# https://www.nature.com/articles/s41531-025-00999-w

#MeanSpeed, MeanRMSVelocity, MeanOpeningSpeed, MeanClosingSpeed
MovementSpeed = np.array([ 0,  0.        ,  1,  0.        ,  1,
        0.        ,  1,  0.        ,  1 ,  0.        ,
        0,  0.        ,  0,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 0,
        0.        ,  0.        ,  0.        ,  0.        , 0])
Data_z['MovementSpeed'] = Data_z[features].dot(MovementSpeed)
#StdMeanSpeed, StdRMSVelocity, StdOpeningSpeed, StdClosingSpeed
PerformanceConsistency = np.array([ 0,  0.        ,  0,  1        ,  0,
        0.        ,  1,  0.        ,  0 ,  1        ,
        0,  1        ,  0,  1       ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , 0,
        0.        ,  0.        ,  0.        ,  0.        , 0])
Data_z['PerformanceConsistency'] = Data_z[features].dot(PerformanceConsistency)


#MeanAmplitude, MeanCycleDuration, StdCycleDuration, and Frequency
MovementTimingandFrequency = np.array([1, 0, 0, 0, 0, 0,
 0, 0, 0, 0, 0,
 0, 0, 0, 1,
 1, 0, 0, 0, 0, 0,
0, 0, 0, -1, 0,
0, 0, 0, 0])
Data_z['MovementTimingandFrequency'] = Data_z[features].dot(MovementTimingandFrequency)

In [14]:
#compare some selected features using linear mixedeffects model

# The model will include the following covariates:
# - Time (V1 or V2)
# - Age (in years)
# - Sex (male or female)
# - Handedness (left or right)
# - BaselineDiseaseDuration (in months)
# - BaselineDiseaseSeverity (baseline UPDRS III score#)

# Filter for subjects with both V1 and V2 visits
v1 = Data_z[Data_z['Visit'] == 'V1']
v2 = Data_z[Data_z['Visit'] == 'V2']
common_subjects = set(v1['Subject_ID']) & set(v2['Subject_ID'])

v1_matched = v1[v1['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
v2_matched = v2[v2['Subject_ID'].isin(common_subjects)].sort_values('Subject_ID')
# Combine v1_matched and v2_matched for mixed model analysis
combined = pd.concat([v1_matched, v2_matched], ignore_index=True)


selected_features = ['MovementSpeed', 'PerformanceConsistency', 'MovementTimingandFrequency',]

for feat in selected_features:
    var = feat
    model = smf.mixedlm(f"{var} ~ Time + Age + Sex + Handedness + BaselineDiseaseDuration + BaselineDiseaseSeverity ", combined, groups=combined["Subject_ID"])   
    result = model.fit()
    print(f"Mixed model results for {var}:")
    print(f"V1 mean ± std: {v1_matched[f'{var}'].mean():.2f} ± {v1_matched[f'{var}'].std():.2f}")
    print(f"V2 mean ± std: {v2_matched[f'{var}'].mean():.2f} ± {v2_matched[f'{var}'].std():.2f}")
    # print(result.summary())

    # Store p-values for Time coefficient
    if 'Time' in result.pvalues.index:
        pval_time = result.pvalues['Time']
    else:
        pval_time = np.nan

    if 'Time' in result.params.index:
        coef_time = result.params['Time']
    else:
        coef_time = np.nan

    if feat == selected_features[0]:
        pvals_time = []
        mean_diffs = []
        std_diffs = []
        percent_diffs = []

    # Compute %difference between v1 and v2
    v1_vals = v1_matched[var].values
    v2_vals = v2_matched[var].values
    diff = (v2_vals - v1_vals) / v1_vals * 100
    mean_diff = np.mean(diff)
    std_diff = np.std(diff)

    pvals_time.append(pval_time)
    mean_diffs.append(mean_diff)
    std_diffs.append(std_diff)
    percent_diffs.append(diff)

# After loop, apply correction and print summary
if feat == selected_features[-1]:
    reject, pvals_corr, _, _ = multipletests(pvals_time, method='fdr_tsbh')
    print("\nCorrected p-values for Time coefficient (FDR):")
    for f, p, pc in zip(selected_features, pvals_time, pvals_corr):
        print(f"{f}: raw p={p:.4g}, corrected p={pc:.4g}")


Mixed model results for MovementSpeed:
V1 mean ± std: -0.25 ± 3.45
V2 mean ± std: -1.55 ± 3.74
Mixed model results for PerformanceConsistency:
V1 mean ± std: 1.01 ± 4.34
V2 mean ± std: 1.70 ± 4.96
Mixed model results for MovementTimingandFrequency:
V1 mean ± std: -0.20 ± 2.34
V2 mean ± std: 0.28 ± 2.94

Corrected p-values for Time coefficient (FDR):
MovementSpeed: raw p=0.00997, corrected p=0.01994
PerformanceConsistency: raw p=0.5215, corrected p=0.3477
MovementTimingandFrequency: raw p=0.2972, corrected p=0.2972
